In [1]:
import jieba
from jieba import posseg
from collections import Counter
import re,time,sys ,os
from simple_io import IO
import math
import requests,json
import datetime


appid = '411654863'         
input_kws = '彩票足球开奖六合彩体育直播篮球，彩六合双色球快比分助手北京福彩预测，全民竞猜三福利App体彩天天中，七星赛车足彩3福利彩票重庆中国球探大师3d吧，博彩足球彩票pk10新浪选五'
split_sig = '，'       #####  关键词之间要以中文逗号分隔


In [2]:
t1 = time.time()

In [3]:
input_kwslist = input_kws.split(split_sig)

def jieba_cut(subtitle):
    seg_list = jieba.cut_for_search(subtitle)
    keyword = []
    for word in seg_list:
        keyword.append(word)
    return keyword

keywords = []
for words in input_kwslist:
    keywords.extend(jieba_cut(words))


Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.839 seconds.
Prefix dict has been built succesfully.


In [4]:
timeout = 4

def rank(word):
    url ='http://rest.appbk.com/word/get_word_hot_rank?n='+ word +'&cc=cn'
    try:
        resp_ = requests.get(url,timeout=timeout,verify = False)
        resp_2 = resp_.content
        resp_3 = resp_2.decode('utf-8')

        resp_list = json.loads(resp_3)
        return int(resp_list['rank'])
    except Exception as e:
        print(e)    
        return -1

In [5]:
wordredudict = {}
for word in keywords:
    wordredudict[word] = rank(word)

In [6]:
for key,value in wordredudict.items():
    if value > 4604:
        if value > 5000:
            qv = 2.5 * math.exp(0.001 * value)
        else:
            qv = value - 4604

        score = math.log(qv) / math.pow(value,0.75)
        wordredudict[key] = score
    
wordlist = sorted(wordredudict.items(),key = lambda x:x[1],reverse = True)    

In [7]:
top70 = []
for item in wordlist[:int(len(wordlist) * 0.7)]:
    top70.append(item[0])

top70str = ''
strlen = 0
for item in top70:
    top70str += item
    strlen += len(item)
    if strlen > 10:
        strlen = 0
        top70str += split_sig
# top70str

In [8]:
##############   语义拓展

word = top70[0]

today = datetime.date.today().strftime('%Y-%m-%d')
yesterday = (datetime.date.today() + datetime.timedelta(days=-1)).strftime('%Y-%m-%d')
now_time_H = datetime.datetime.now().strftime('%H')

timeout = 4


In [9]:
def competitive_product(word,appnum):
    url ='http://rest.appbk.com/app/get_all_app_search_results?n=' + word + '&start=0&limit=100&cc=cn' 
    appid = []
    try:
        resp_ = requests.get(url,timeout=timeout,verify = False)
        resp_2 = resp_.content
        resp_3 = resp_2.decode('utf-8')
        
        resp_list = json.loads(resp_3)
        resp_list = resp_list['results']
        for item in resp_list:
            appid.append(item['app_id']) 
            if len(appid) > appnum:
                break
        return appid    
    except Exception as e:
        print(e)

In [10]:
def competitive_product_kws_rank(appid_list,kwsnum):
    kws = []
    rank = []
    for appid in appid_list:       
        url ='http://rest.appbk.com/word/get_app_word_two_date_compare?email=&app_id=' + appid + '&token=&start=0&limit=12000&simple=1&cur_date=' + today + '&pre_date=' + yesterday        
        try:
            resp_ = requests.get(url,timeout=timeout,verify = False)
            resp_2 = resp_.content
            resp_3 = resp_2.decode('utf-8')

            resp_list = json.loads(resp_3)
            resp_list = resp_list['results']

            for item in resp_list:
                if item[0] not in kws and int(item[3])>4604:
                    kws.append(item[0]) 
                    rank.append(int(item[3])) 
                    if len(kws) > kwsnum:
                        break  
        except Exception as e:
            print(e)
    res = dict(zip(kws, rank))
    
    return res

In [11]:
### 输入词下的前15个竞品appid
appid = competitive_product(word,10)

### 所有竞品关键词，每个竞品取热度前100的关键词
wordredudict = competitive_product_kws_rank(appid,100)


In [12]:
###### 生成后30%的关键词

wordlist = sorted(wordredudict.items(),key = lambda x:x[1],reverse = True)
for item in wordlist:
    seg_list = posseg.cut(item[0])
    for word,flag in seg_list:
        if flag == 'n':
            if word not in top70:
                top70.append(word)
                top70str += word
                strlen += len(word)
                if strlen > 10:
                    strlen = 0
                    top70str += split_sig
                if len(top70str) > 100:
                    break
top70str = top70str[:100]
top70str

'色球彩票足彩足球体彩直播，福利彩票足球彩票球探六合彩，pk10赛车体育双色球，博彩竞猜篮球福彩天天新浪，比分六合3d福利预测大师，彩App重庆酷彩麻将校友，车网医程机网软件彩站人人，盈先锋结果官方助手'

In [13]:
time.time()-t1

15.689949035644531